# Calculates the energy for 2 ions under crystal field, given (Bx,By,Bz) and dipolar interaction

### Uses:
* Compare $\langle J^z \rangle$ vs. $H_x$ for 2 ions with and without $xx$, $yy$, $xy$, $yx$ terms of the dipolar interaction
* Check (using curve fitting) the functional dependence of $\langle J^z \rangle$ vs. $H_x$. Compare parabolic vs. quadratic.

In [1]:
import numpy as np
from numpy import linalg as LA
import math
import sys
from matplotlib import pyplot as plt
import itertools

In [2]:
%matplotlib qt

In [2]:
#constants
hbar = 1
J=8
deg_J = 2 * J + 1
g_L = 5/4 # Lande g-factor
# u_B = 0.6717 # Bohr magneton
a=5.175*10**-10
c=10.75*10**-10
# mu0_over_4pi=10**-7
mu0=0.2428284177e4
u_B=0.3462955026e-4
# k_B=1.380649*10**-23
k_B=0.0515540067e-3

In [3]:
# J matrices
jplus = hbar * np.diag(np.array( [ math.sqrt(J*(J+1) - m*(m+1)) for m in np.arange(-J,J) ] ), 1)
jminus = hbar * np.diag(np.array( [ math.sqrt(J*(J+1) - m*(m-1)) for m in np.arange(-J+1,J+1) ] ),-1)
jx = (jplus + jminus) * 0.5
jy = (jplus - jminus) * (-0.5j)
jz = hbar * np.diag(np.arange(-J,J+1))
I_J = np.diag(np.ones(int(round(deg_J))))

In [4]:
# crystal field equivalent operators
O02 = 3 * LA.matrix_power(jz,2) - J*(J+1)*I_J
O04 = 35 * LA.matrix_power(jz,4) - 30 * J * (J+1) * LA.matrix_power(jz,2) + 25*LA.matrix_power(jz,2) - 6 * J * (J+1) * I_J + 3 * J**2 * (J+1)**2 * I_J
O44C = 0.5 * (LA.matrix_power(jplus,4) + LA.matrix_power(jminus,4))
O06 = 231 * LA.matrix_power(jz,6) - 315*J*(J+1)*LA.matrix_power(jz,4) + 735*LA.matrix_power(jz,4) + 105 * J**2 * (J+1)**2 * LA.matrix_power(jz,2) - 525*J*(J+1)*LA.matrix_power(jz,2) + 294*LA.matrix_power(jz,2) - 5 * J**3 * (J+1)**3 * I_J + 40 * J**2 *(J+1)**2 * I_J - 60*J*(J+1)*I_J
O46C1 = 0.25 * (LA.matrix_power(jplus,4) + LA.matrix_power(jminus,4)) @ (11*LA.matrix_power(jz,2) - J*(J+1)*I_J - 38*I_J)
O46S1 = -0.25j * (LA.matrix_power(jplus,4) - LA.matrix_power(jminus,4)) @ (11 * LA.matrix_power(jz,2) - J*(J+1)*I_J - 38*I_J)
O46C = O46C1 + np.transpose(np.conj(O46C1))
O46S = O46S1 + np.transpose(np.conj(O46S1))

In [5]:
# crystal field parameters
B02 = -0.696
B04 = 4.06e-3
B06 =  4.64e-6
B44C = 0.0418
B46C = 8.12e-4
B46S = 1.137e-4

In [6]:
# crystal field Hamiltonian
H_cf = B02*O02 + B04*O04 + B06*O06 + B44C*O44C + B46C*O46C + B46S*O46S

## Single run: give (Bx,By,Bz)

In [106]:
# create dipolar interaction
def dipolar_int(r, m, n):
    return ((1 if m==n else 0)*LA.norm(r)**2 - 3*(r[m])*(r[n]))/LA.norm(r)**5

# relative location of the two ions
r=[a/2,0,c/4]

In [7]:
# int_table = np.array([[-4.571443122909957E-19,-4.320262326949886E-19,2.9835473291874735],
#                         [1.9548866539632312E-16,8.422043478184144E-17,-4.3116276077959],
#                         [2.724444833807359E-17,2.977950427134481E-17,-5.631301873053044],
#                         [-3.693130513476026E-17,5.749197707653294E-16,-4.3116276077959],
#                         [-4.571443122909957E-19,-4.320262326949886E-19,2.9835473291874735],
#                         [-2.010353222792296E-17,2.2554084800166757E-16,-4.311627607795901],
#                         [2.724444833807359E-17,2.977950427134481E-17,-5.631301873053044],
#                         [-4.571443122909957E-19,-4.320262326949886E-19,2.9835473291874735],
#                         [1.9549333285205303E-16,8.422043478184144E-17,-4.3116276077959],
#                         [-4.571443122909957E-19,-4.320262326949886E-19,2.9835473291874735]])
int_table = np.array([[-0.0000000000,-0.0000000000,2.9835473292],
[0.0000000000,0.0000000000,-4.3116276078],
[0.0000000000,0.0000000000,-5.6313018731],
[-0.0000000000,0.0000000000,-4.3116276078],
[-0.0000000000,-0.0000000000,2.9835473292],
[-0.0000000000,0.0000000000,-4.3116276078],
[0.0000000000,0.0000000000,-5.6313018731],
[-0.0000000000,-0.0000000000,2.9835473292],
[0.0000000000,0.0000000000,-4.3116276078],
[-0.0000000000,-0.0000000000,2.9835473292]])
res=[]
for mu in range(3):
    b=np.zeros((4,4))
    b[np.triu_indices(4)]=int_table[:,mu]
    res.append(np.tril(b.T,-1) + b)
int_table = np.array(res)

In [8]:
from scipy.sparse import csr_matrix,kron
from scipy.sparse.linalg import eigsh

In [9]:
H_cf_sparse=csr_matrix(H_cf)
I_J_sparse=csr_matrix(I_J)

In [10]:
def tensor_prod(A, B, C, D):
    return kron(kron(kron(A,B,format='csr'),C,format='csr'),D,format='csr')

In [11]:
H_cf4 = (tensor_prod(H_cf_sparse, I_J_sparse, I_J_sparse, I_J_sparse) 
          + tensor_prod(I_J_sparse, H_cf_sparse, I_J_sparse, I_J_sparse) 
          + tensor_prod(I_J_sparse, I_J_sparse, H_cf_sparse, I_J_sparse) 
          + tensor_prod(I_J_sparse, I_J_sparse, I_J_sparse, H_cf_sparse))

In [19]:
H_int4 = csr_matrix((83521, 83521))
ions = {0 : [tensor_prod(jx,I_J,I_J,I_J), tensor_prod(jy,I_J,I_J,I_J), tensor_prod(jz,I_J,I_J,I_J)],
       1 : [tensor_prod(I_J,jx,I_J,I_J), tensor_prod(I_J,jy,I_J,I_J), tensor_prod(I_J,jz,I_J,I_J)],
       2 : [tensor_prod(I_J,I_J,jx,I_J), tensor_prod(I_J,I_J,jy,I_J), tensor_prod(I_J,I_J,jz,I_J)],
       3 : [tensor_prod(I_J,I_J,I_J,jx), tensor_prod(I_J,I_J,I_J,jy), tensor_prod(I_J,I_J,I_J,jz)]}
for i in range(4):
    for j in range(4):
#         ((1/a**3)*k_B*mu0_over_4pi*
        H_int4 += ( ((mu0*0.25/math.pi)/k_B)*(g_L*u_B)**2 * 
                   (0.5 if i==j else 1)*(int_table[2,i,j]*ions[i][2].multiply(ions[j][2])             # zz
                    + int_table[0,i,j]*ions[i][0].multiply(ions[j][2])           # xz
                       + int_table[0,i,j]*ions[i][2].multiply(ions[j][0])        # zx
                    + int_table[1,i,j]*ions[i][1].multiply(ions[j][2])           # yz
                       + int_table[1,i,j]*ions[i][2].multiply(ions[j][1])   ) )  # zy
#         print('(%s,%s) : %s'%(i,j,(1/a**3)*k_B*mu0_over_4pi*(g_L*u_B)**2 * 
#                    int_table[2,i,j]))
H_int4*=0.5

In [20]:
i=1
j=2
(0.5 if i==j else 1)

1

In [22]:
Bx = 0.0
By = 0.0
Bz = 0.0
H_zeeman4 = csr_matrix((83521, 83521))
for i in range(4):
    H_zeeman4 += -g_L* 0.6717 * (Bx * ions[i][0] + By * ions[i][1] + Bz * ions[i][2])
H_full4 = H_cf4 + H_int4 + H_zeeman4
eigenvalues, eigenstates = eigsh(H_full4,k=16,which='SA')
beta=1/1.53
Jz = ions[0][2] + ions[1][2] + ions[2][2] + ions[3][2]
energy = np.average(eigenvalues,weights=np.exp(-beta*eigenvalues))
magnetization = np.average(np.diagonal(np.conj(eigenstates.T) @ Jz @ eigenstates),weights=np.exp(-beta*eigenvalues))
print(Bx, energy/4)
print(Bx, magnetization)

0.0 -247.20689545754783
0.0 (-0.03524838952036259-1.161621198677157e-17j)


In [31]:
magnetization = np.average(np.diagonal(np.conj(eigenstates.T) @ Jz @ eigenstates),weights=np.exp(-beta*eigenvalues))

In [32]:
magnetization

(0.02663566656879611-1.2457539033678776e-16j)

In [ ]:
# Magnetic field Zeeman term
Bx = 0.0
By = 0.0
Bz = 0.0

Jz = ions[0][2] + ions[1][2] + ions[2][2] + ions[3][2]

res_energy=[]
res_magnetization=[]

for bx in np.linspace(0,2):
    H_zeeman4 = csr_matrix((83521, 83521))
    for i in range(4):
        H_zeeman4 += -g_L*u_B * (bx * ions[i][0] + By * ions[i][1] + Bz * ions[i][2])
    H_full4 = H_cf4 + H_int4 + H_zeeman4
    eigenvalues, eigenstates = eigsh(H_full4,k=16,which='SA')
    beta=1/1.53
    energy = np.average(eigenvalues,weights=np.exp(-beta*eigenvalues))
    magnetization = np.average(np.real(np.diagonal(np.conj(eigenstates.T) @ Jz @ eigenstates)),weights=np.exp(-beta*eigenvalues))
    res_energy.append(energy/4)
    res_magnetization.append(magnetization)
    print(bx, energy/4, np.real(magnetization))
res_energy=np.array(res_energy)
res_magnetization=np.array(res_magnetization)

In [ ]:
np.savez('energy.npz',np.linspace(0,2),res)

In [173]:
mc_res=np.array([[0.0, -247.218],
[0.04081633, -247.193],
[0.08163265, -247.181],
[0.12244898, -247.207],
[0.16326531, -247.18],
[0.20408163, -247.222],
[0.24489796, -247.211],
[0.28571429, -247.22]])

In [171]:
mc_res[:,0]

array([0.        , 0.04081633, 0.08163265, 0.12244898, 0.16326531])

In [ ]:
plt.plot(np.linspace(0,2),res,marker='o',linestyle='none',fillstyle='none')
plt.plot(mc_res[:,0],mc_res[:,1], marker='D',linestyle='none',fillstyle='none')

In [33]:
H_full4 = H_cf4 + H_int4 + H_zeeman4

In [41]:
eigenvalues = eigsh(H_full4,which='SA')[0]

In [42]:
beta=1/1.53
energy = np.average(eigenvalues,weights=np.exp(-beta*eigenvalues))

In [43]:
energy/4

-245.96061122135228

## Plot energy levels

In [13]:
# total spin
plt.plot(np.real(np.diagonal(np.conj(v.T)@(np.kron(I_J,jz)+np.kron(jz,I_J))@v))[:4],w[:4],'k_')
plt.show()

In [14]:
# one ion spin
plt.plot(np.real(np.diagonal(np.conj(v.T)@(np.kron(I_J,jz))@v))[:4],w[:4],'k_')
plt.show()

In [15]:
# other ion spin
plt.plot(np.real(np.diagonal(np.conj(v.T)@(np.kron(jz,I_J))@v))[:4],w[:4],'k_')
plt.show()

## Calculate & plot $\langle J^z \rangle$ vs. $H_x$ with and without $xx$, $yy$, etc. terms

In [146]:
res_energy_up=[]
res_energy_down=[]
res_magnetic_moment_up=[]
res_magnetic_moment_down=[]
all_energies=[]
magnetic_moment_all_int=[]
energies_all_int=[]
magnetic_moment_no_xxyy=[]
energies_no_xxyy=[]
energies_up_single=[]
energies_down_single=[]
magnetic_moments_up_single=[]
magnetic_moments_down_single=[]
SxSz_exp_val1=[]
SxSz_exp_val2=[]

Bx=np.linspace(-.5,.5,num=300)
for bx in Bx:
    H_zeeman = u_B*g_L*(bx*jx + By*jy + Bz*jz)    # zeeman term
    H1 = H_cf2 + H_int_all - np.kron(H_zeeman,I_J) - np.kron(I_J,H_zeeman)    # full hamiltonian
    H2 = H_cf2 + H_int_no_xxyy - np.kron(H_zeeman,I_J) - np.kron(I_J,H_zeeman)    # full hamiltonian
    w,v = LA.eigh(H1)
#     all_energies.append(w)
    # initially assume lower level is up and upper is down
    energy_up_single = w[0]
    energy_down_single = w[1]
    magnetic_moment_up_single = np.real(np.diagonal(np.conj(v.T)@np.kron(I_J,jz)@v)[0])
    magnetic_moment_down_single = np.real(np.diagonal(np.conj(v.T)@np.kron(I_J,jz)@v)[1])
    if (magnetic_moment_up_single < magnetic_moment_down_single):
        # switch moments
        temp = magnetic_moment_up_single
        magnetic_moment_up_single = magnetic_moment_down_single
        magnetic_moment_down_single = temp
        # switch energies
        temp = energy_up_single
        energy_up_single = energy_down_single
        energy_down_single = temp
    energies_up_single.append(energy_up_single)
    energies_down_single.append(energy_down_single)
    magnetic_moments_up_single.append(magnetic_moment_up_single)
    magnetic_moments_down_single.append(magnetic_moment_down_single)
    magnetic_moment_all_int.append(np.real(np.diagonal(np.conj(v.T)@(np.kron(I_J,jz)+np.kron(jz,I_J))@v)))
    #+np.kron(jz,I_J)
    energies_all_int.append(w)
    w,v = LA.eigh(H2)
    magnetic_moment_no_xxyy.append(np.real(np.diagonal(np.conj(v.T)@(np.kron(I_J,jz)+np.kron(jz,I_J))@v)))
    energies_no_xxyy.append(w)
    SxSz_exp_val1.append(np.real(np.diagonal(np.conj(v.T)@(np.kron(jx,I_J))@v)))
    SxSz_exp_val2.append(np.real(np.diagonal(np.conj(v.T)@(np.kron(jz,I_J))@v)))
    
SxSz_exp_val1 = np.array(SxSz_exp_val1)
SxSz_exp_val2 = np.array(SxSz_exp_val2)
magnetic_moments_up_single = np.array(magnetic_moments_up_single)
magnetic_moments_down_single = np.array(magnetic_moments_down_single)
energies_up_single = np.array(energies_up_single)
energies_down_single = np.array(energies_down_single)
energy_up_arr = np.array(res_energy_up)
energy_down_arr = np.array(res_energy_down)
magnetic_moment_up_arr = np.array(res_magnetic_moment_up)
magnetic_moment_down_arr = np.array(res_magnetic_moment_down)
magnetic_moment_all_int = np.array(magnetic_moment_all_int)
magnetic_moment_no_xxyy = np.array(magnetic_moment_no_xxyy)
energies_all_int = np.array(energies_all_int)
energies_no_xxyy = np.array(energies_no_xxyy)

In [116]:
# plotting

# fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
level=4
# lineObjects_no_xxyy = plt.plot(Bx,magnetic_moment_no_xxyy[:,0]);
# lineObjects_all_int = plt.plot(Bx,magnetic_moment_all_int[:,0],'--');

# plt.plot(Bx,(magnetic_moment_no_xxyy[:,:4]-magnetic_moment_all_int[:,:4])/magnetic_moment_no_xxyy[:,:4])
lineObjects_all_int = plt.plot(Bx,energies_all_int[:,:2])
lineObjects_no_xxyy = plt.plot(Bx,energies_no_xxyy[:,:2])
# plt.legend(iter(lineObjects_all_int+lineObjects_no_xxyy), itertools.product(['+','-'],range(level)))
# plt.hlines(magnetic_moment_no_xxyy[-1,:4],-1,1.1)
plt.ylabel(r'<$J_z$>');
plt.xlabel(r'$H_x$');
# np.max((magnetic_moment_no_xxyy[:,:4]-magnetic_moment_all_int[:,:4])/magnetic_moment_no_xxyy[:,:4])
# plt.plot(Bx,magnetic_moment_up_arr);

### Find peak value

In [112]:
# used to find the peak locattion of the <J_z> vs. H_x curve.
# when the two ions are directly one above the other the peak should be at H_x=0
# when the two ions are offset along, say, the x-axis, the peak moves slightly.
# this movement should correspond to the effective transverse magnetic field exerted one on the other by the two ions.
# this cell find the peak location.
# when run with r=[0,0,c] the printed value should be effectively zero.
# when run with r=[a,0,c] the printed value should be equal to the effective field, calculated in the next cell.
# notice the Bx range must be set such that the peak is actually included!

from scipy.interpolate import interp1d
f_all_int = interp1d(Bx, magnetic_moment_all_int[:,0], kind = 'cubic')
Bx_new = np.linspace(Bx[0], Bx[-1],1000000)
print(Bx_new[np.argmax(f_all_int(Bx_new))])

-0.17800367800367806


In [90]:
# plot peak
plt.plot(Bx_new, f_all_int(Bx_new))
plt.plot(Bx_new[np.argmax(f_all_int(Bx_new))],f_all_int(Bx_new[np.argmax(f_all_int(Bx_new))]), 'rx')
plt.grid()
plt.xlabel(r'$H_x$');
plt.ylabel(r'<$J_z$>');

### Internal transverse field

In [114]:
# internal transverse field exerted by each of the ions on the other ion.
-5.51*k_B*mu0_over_4pi*g_L*u_B*(-3*r[0]*r[2])/LA.norm(r)**5

0.18439233988686662

## Fit parabola and quadratic function to $\langle J^z \rangle$ vs. $H_x$ curve

In [68]:
def parabola(x, a, c):
    return a * x**2 + c
def quad(x, a, c):
    return a * x**4 + c

In [69]:
def r_squared(xdata, ydata, f, popt):
    residuals = ydata- f(xdata, *popt)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    
    return r_squared

In [70]:
from scipy.optimize import curve_fit
ydata=magnetic_moment_all_int[:,0]
xdata=Bx
poptquad, pcovquad = curve_fit(quad, xdata, ydata,p0=[1,11])
poptparab, pcovparab = curve_fit(parabola, xdata, ydata,p0=[1,11])

print('quad fit R^2 = %5.3f' % r_squared(xdata, ydata, quad, poptquad))
print('parabola fit R^2 = %5.3f' % r_squared(xdata, ydata, parabola, poptparab))

quad fit R^2 = 0.382
parabola fit R^2 = 0.394


In [24]:
# plot
plt.plot(xdata, ydata, 'b.', label='data')
plt.plot(xdata, quad(xdata, *poptquad), 'g-',label=r'$a x^4 + c$ fit: a=%5.3f, c=%5.3f ($R^2$=%5.3f)' % tuple(list(poptquad)+[r_squared(xdata, ydata, quad, poptquad)]))
plt.plot(xdata, parabola(xdata, *poptparab), 'r-',label=r'$a x^2 + c$ fit: a=%5.3f, c=%5.3f ($R^2$=%5.3f)' % tuple(list(poptparab)+[r_squared(xdata, ydata, parabola, poptparab)]))
plt.xlabel(r'$H_x$')
plt.ylabel(r'<$J_z$>');
plt.legend();

## How to express $S_i^z S_j^x$

In [137]:
# Energy like it's calculated in the simulation
magnetic_moments_up_single

AttributeError: 'list' object has no attribute 'size'

In [129]:
(-5.51*k_B*mu0_over_4pi*g_L*u_B*(-3*r[0]*r[2])/LA.norm(r)**5) * magnetic_moments_

0.18439233988686662

In [147]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel(r'$H_x$')
ax1.set_ylabel(r'$\langle J_x \rangle$', color=color)
ax1.plot(Bx, SxSz_exp_val1[:,0], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel(r'$\langle J_z \rangle$', color=color)  # we already handled the x-label with ax1
ax2.plot(Bx, SxSz_exp_val2[:,0], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

